In [1]:
import pandas as pd
import numpy as np

In [13]:
df1 = pd.read_csv('subtaskA_test_Ire.csv').drop(columns=['Id']).rename(columns={'conspiratorial': 'Annotator_1'})
df2 = pd.read_csv('subtaskA_test_marti.csv').drop(columns=['Id']).rename(columns={'conspiratorial': 'Annotator_2'})
df3 = pd.read_csv('subtaskA_clean_test_cased_labeled_Elena.csv',sep=';').rename(columns={'label': 'Annotator_3'})

In [14]:
df1.head()

,comment_text,Annotator_1
0,La CNN ha appena ammesso che Trump non è più a...,1
1,"Notizia del 2017: “Autovelox, la foto viola la...",0
2,"La foto di due ""sospetti"" in un palazzo non è ...",0
3,La falsa notizia della bambina con il cartello...,0
4,Ah dimenticavo.. e' gratuita,0


In [15]:
df = df1.copy()
df['Annotator_2'] = df2['Annotator_2']
df['Annotator_3'] = df3['Annotator_3']

In [16]:
df.head()

,comment_text,Annotator_1,Annotator_2,Annotator_3
0,La CNN ha appena ammesso che Trump non è più a...,1,1,1
1,"Notizia del 2017: “Autovelox, la foto viola la...",0,0,0
2,"La foto di due ""sospetti"" in un palazzo non è ...",0,0,0
3,La falsa notizia della bambina con il cartello...,0,0,0
4,Ah dimenticavo.. e' gratuita,0,0,0


In [6]:
df.shape

(460, 4)

random perturbation

In [17]:
#for i in range(151):
  #df.loc[np.random.randint(10), 'Annotator_2'] = np.random.randint(2)
  #df.loc[np.random.randint(10), 'Annotator_3'] = np.random.randint(2)


In [18]:
df.head()

,comment_text,Annotator_1,Annotator_2,Annotator_3
0,La CNN ha appena ammesso che Trump non è più a...,1,1,1
1,"Notizia del 2017: “Autovelox, la foto viola la...",0,0,0
2,"La foto di due ""sospetti"" in un palazzo non è ...",0,0,0
3,La falsa notizia della bambina con il cartello...,0,0,0
4,Ah dimenticavo.. e' gratuita,0,0,0


## random perturbation
non runnare on actual data

In [41]:
for i in range(11):
  df.loc[np.random.randint(df.shape[0]+1), 'Annotator_1'] = np.random.randint(2,3)
  df.loc[np.random.randint(df.shape[0]+1), 'Annotator_2'] = np.random.randint(2,3)
  df.loc[np.random.randint(df.shape[0]+1), 'Annotator_3'] = np.random.randint(2,3)

# cleaning

In [19]:
def check_vals(unique_vals_exp, df, columns, return_dict=False):
  idxs = dict()
  for col in df.columns[1:]:
    unique_vals = np.unique(df[col])
    if not np.array_equal(unique_vals, unique_vals_exp):
      print('Errors found in col', col)
      print('Indexes of errors:', list(df.query(f'{col} not in {unique_vals_exp}').index))
      idxs[col] = list(df.query(f'{col} not in {unique_vals_exp}').index)

  if len(idxs) == 0:
      print('no errors found')

  if return_dict:
    return idxs


In [20]:
unique_vals_exp = [0,1]
check_vals(unique_vals_exp, df, df.columns[1:])

no errors found


In [44]:
idx_dict = check_vals(unique_vals_exp, df, df.columns[1:], return_dict=True)
for col, idx in idx_dict.items():
  df.loc[idx, col] = 1

Errors found in col Annotator_1
Indexes of errors: [131, 239, 329, 578, 712, 918, 1161, 1359, 1466, 1753, 1828]
Errors found in col Annotator_2
Indexes of errors: [73, 99, 150, 446, 534, 650, 678, 1172, 1426, 1597, 1727]
Errors found in col Annotator_3
Indexes of errors: [194, 434, 635, 653, 726, 729, 962, 1106, 1254, 1536, 1669]


In [45]:
check_vals(unique_vals_exp, df, df.columns[1:])

no errors found


# Gold Standard

In [21]:
df['Gold_Standard'] = 42
for idx ,row in df[['Annotator_1','Annotator_2','Annotator_3']].iterrows():
  vals = np.array([row['Annotator_1'], row['Annotator_2'], row['Annotator_3']])
  df.loc[idx, 'Gold_Standard'] = np.argmax(np.bincount(vals))

In [22]:
df.head()

,comment_text,Annotator_1,Annotator_2,Annotator_3,Gold_Standard
0,La CNN ha appena ammesso che Trump non è più a...,1,1,1,1
1,"Notizia del 2017: “Autovelox, la foto viola la...",0,0,0,0
2,"La foto di due ""sospetti"" in un palazzo non è ...",0,0,0,0
3,La falsa notizia della bambina con il cartello...,0,0,0,0
4,Ah dimenticavo.. e' gratuita,0,0,0,0


# Inter-annotator agreement

### **K = P(A) - P(E) / 1 - P(E)**

P(A) = Volte in cui gli annotatori si sono mostrati d'accordo

P(E) = Volte in cui ci si aspetta che gli annotatori per caso compiano la stessa scelta (e = expected)

The higher the better

In [23]:
def interannotator_agreement(annotator1, annotator2):
  num_records = len(annotator1)
  label1, label2 = np.unique(annotator1)

  counts1 = np.unique(annotator1, return_counts=True)[1]
  counts2 = np.unique(annotator2, return_counts=True)[1]
  annotation_prob = {
      'Annotator_1': {
          'label1': counts1[0]/num_records,
          'label2': counts1[1]/num_records
      },
      'Annotator_2': {
          'label1': counts2[0]/num_records,
          'label2': counts2[1]/num_records
      }
  }

  alignment = 0
  for val1, val2 in zip(annotator1, annotator2):
    if val1 == val2:
      alignment +=1

  p_a = alignment/num_records
  p_e = (annotation_prob['Annotator_1']['label1'] * \
        annotation_prob['Annotator_2']['label1']) + \
        (annotation_prob['Annotator_1']['label2'] * \
        annotation_prob['Annotator_2']['label2'])
  k =  (p_a - p_e) / (1 - p_e)

  return k



In [24]:
k1 = interannotator_agreement(np.array(df['Annotator_1']), np.array(df['Annotator_2']))
print('k1:', k1)

k1: 0.5388563637821799


In [25]:
from statistics import harmonic_mean, mean

k1 = interannotator_agreement(np.array(df['Annotator_1']), np.array(df['Annotator_2']))
k2 = interannotator_agreement(np.array(df['Annotator_1']), np.array(df['Annotator_3']))
k3 = interannotator_agreement(np.array(df['Annotator_2']), np.array(df['Annotator_3']))
agg = (k1+k2+k3)/3

print('k1:', k1)
print('k2:', k2)
print('k3:', k3)
print('Mean:', agg)



k1: 0.5388563637821799
k2: 0.5534341881724779
k3: 0.6727174832705177
Mean: 0.5883360117417252
